In [1]:
!pip install transformers datasets evaluate accelerate scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
model = AutoModelForSequenceClassification.from_pretrained(
    'neuralmind/bert-base-portuguese-cased',
    num_labels=2
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [6]:
df=pd.read_csv('/content/gdrive/MyDrive/sslr/supervised.csv')

In [7]:
df.head()

,text,author,is_machado,source_file,chunk_id
0,rosto dois grandes olhos azuis e sacudindo ao ...,Machado de Assis,1,machado_de_assis/raw/txt/conto/contosFluminens...,1
1,suspiro a figura é poética mas não é a da hero...,Machado de Assis,1,machado_de_assis/raw/txt/conto/contosFluminens...,2
2,quer dizer simplesmente que a rapariga é rica ...,Machado de Assis,1,machado_de_assis/raw/txt/conto/contosFluminens...,4
3,dollar estava aboletada na casa de um sujeito ...,Machado de Assis,1,machado_de_assis/raw/txt/conto/contosFluminens...,6
4,a doença e tão excelente era o elixir que o au...,Machado de Assis,1,machado_de_assis/raw/txt/conto/contosFluminens...,8


In [8]:
df['is_machado'].value_counts()

,count
is_machado,
1,7581
0,7540


In [9]:
# Prepare the dataset
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_length)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Split the data
X_train, X_val, y_train, y_val = train_test_split(df['text'], df['is_machado'], test_size=0.2)

# Create datasets
train_dataset = TextDataset(X_train.tolist(), y_train.tolist(), tokenizer)
val_dataset = TextDataset(X_val.tolist(), y_val.tolist(), tokenizer)


In [10]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate all metrics
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


In [11]:
import shutil
import os


In [12]:
# Learning rates to test
learning_rates = [1e-5, 2e-5, 3e-5, 4e-5, 5e-5]
results = []
batch_size = 16
eval_batch_size = 32
num_train_epochs = 4

for lr in learning_rates:
    print(f"\nTraining with learning rate: {lr}")
    
    output_dir = f'/content/gdrive/MyDrive/sslr/results_lr_{lr}'
    
    # Clear previous run's directory if it exists
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    
    model = AutoModelForSequenceClassification.from_pretrained(
        'neuralmind/bert-base-portuguese-cased',
        num_labels=2
    )
    
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=eval_batch_size,
        weight_decay=0.01,
        logging_dir=f'./logs_lr_{lr}',
        learning_rate=lr,
        evaluation_strategy="steps",
        eval_steps=300,
        save_strategy="steps",
        save_steps=300,
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        greater_is_better=True,
        fp16=True,
        report_to=[],
        push_to_hub=False,
        overwrite_output_dir=True
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics
    )
    
    # Train
    train_output = trainer.train()
    
    # Save best model
    trainer.save_model(f'{output_dir}/best_model')
    
    # Cleanup checkpoints
    checkpoint_dirs = [d for d in os.listdir(output_dir) if d.startswith('checkpoint-')]
    for d in checkpoint_dirs:
        shutil.rmtree(os.path.join(output_dir, d))
    
    # Evaluate
    eval_output = trainer.evaluate()
    
    # Store results
    results.append({
        'learning_rate': lr,
        'train_loss': train_output.training_loss,
        'eval_loss': eval_output['eval_loss'],
        'eval_accuracy': eval_output['eval_accuracy'],
        'eval_f1': eval_output['eval_f1']
    })



Training with learning rate: 1e-05


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
300,No log,0.407869,0.807934,0.807986,0.809912,0.807934
600,0.463100,0.333906,0.863140,0.863171,0.865404,0.863140
900,0.463100,0.425866,0.839008,0.838294,0.841604,0.839008
1200,0.314700,0.309100,0.875702,0.875407,0.884818,0.875702
1500,0.266400,0.309562,0.887273,0.887270,0.890500,0.887273
1800,0.266400,0.350500,0.871405,0.871460,0.872050,0.871405
2100,0.219400,0.359507,0.886281,0.886229,0.890788,0.886281
2400,0.219400,0.359711,0.883967,0.883771,0.891602,0.883967
2700,0.183000,0.402427,0.869421,0.869433,0.869455,0.869421
3000,0.173000,0.368649,0.879339,0.879386,0.880822,0.879339



Training with learning rate: 2e-05


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


...

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
300,No log,0.408515,0.836364,0.835575,0.839252,0.836364
600,0.416700,0.302837,0.882314,0.882359,0.883831,0.882314
900,0.416700,0.390805,0.863140,0.863052,0.863210,0.863140
1200,0.297100,0.281257,0.890579,0.890420,0.897717,0.890579
1500,0.243800,0.310467,0.893884,0.893861,0.897733,0.893884
1800,0.243800,0.399942,0.890248,0.890291,0.891708,0.890248
2100,0.174400,0.444526,0.895537,0.895345,0.903695,0.895537
2400,0.174400,0.516733,0.886612,0.886441,0.893847,0.886612
2700,0.131600,0.649159,0.858843,0.858658,0.859221,0.858843
3000,0.106400,0.539556,0.884959,0.885002,0.886543,0.884959



Training with learning rate: 5e-05


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
300,No log,0.352132,0.848595,0.848529,0.848589,0.848595
600,0.421300,0.318349,0.876364,0.876416,0.877575,0.876364
900,0.421300,0.526688,0.857521,0.857309,0.857995,0.857521
1200,0.313700,0.296739,0.893884,0.893754,0.900500,0.893884
1500,0.255200,0.313083,0.896529,0.896557,0.898665,0.896529
1800,0.255200,0.321070,0.896529,0.896565,0.898296,0.896529
2100,0.192400,0.412804,0.893223,0.892935,0.903373,0.893223
2400,0.192400,0.497912,0.892231,0.892182,0.896775,0.892231
2700,0.147300,0.558156,0.873058,0.873003,0.873077,0.873058
3000,0.119200,0.559385,0.875702,0.875738,0.875903,0.875702



Results summary:
   learning_rate  train_loss  eval_loss  eval_accuracy   eval_f1
0        0.00001    0.268754   0.309562       0.887273  0.887270
1        0.00002    0.233877   0.420634       0.886942  0.886947
2        0.00003    0.225628   0.334451       0.889256  0.889262
3        0.00004    0.227074   0.444526       0.895537  0.895345
4        0.00005    0.240121   0.321070       0.896529  0.896565

Best learning rate: 5e-05
Best F1 score: 0.8966


In [ ]:
df_test = pd.read_csv('/content/gdrive/MyDrive/sslr/test.csv')
df_test.head()
